In [2]:
from datetime import datetime
starttime = datetime.now()

In [3]:
elapsedtime = datetime.now() - starttime
print( 'time elapsed (hh:mm:ss.ms {})'.\
      format(elapsedtime))

time elapsed (hh:mm:ss.ms 0:00:00.015956)


In [4]:
def timecheck(starttime):
    elapsedtime = datetime.now() - starttime
    print( 'time elapsed (hh:mm:ss.ms {})'.\
    format(elapsedtime))

# 1. 데이터 불러오기

### 1-1. pandas

In [5]:
import pandas as pd

starttime = datetime.now()

### 0. 데이터 불러오기
selloutDataP = pd.read_csv\
        ("../dataset/kopo_channel_result_small.csv",
           encoding="ms949")
timecheck(starttime)

time elapsed (hh:mm:ss.ms 0:00:02.325776)


### 1-2. Dask

In [6]:
import dask.dataframe as dd

starttime = datetime.now()

### 0. 데이터 불러오기
selloutDataD = dd.read_csv\
        ("../dataset/kopo_channel_result_small.csv",
           encoding="ms949")
timecheck(starttime)

time elapsed (hh:mm:ss.ms 0:00:00.023974)


# 2. 데이터 조회하기

### 2-1. pandas

In [7]:
starttime = datetime.now()
targetColumns = ["REGIONID","PRODUCT","ITEM",\
                 "YEARWEEK","YEAR","QTY"]
selectedDataP = selloutDataP.loc\
                [ selloutDataP.QTY < 0 , targetColumns]
timecheck(starttime)
selectedDataP

time elapsed (hh:mm:ss.ms 0:00:00.014959)


,REGIONID,PRODUCT,ITEM,YEARWEEK,YEAR,QTY
240,A01,PRODUCT12,ITEM01160,201728,2017,-3
1125,A01,PRODUCT34,ITEM01197,201728,2017,-3
1384,A01,PRODUCT34,ITEM02989,201728,2017,-3
1448,A01,PRODUCT34,ITEM00588,201728,2017,-3
2186,A01,PRODUCT12,ITEM02952,201728,2017,-3
...,...,...,...,...,...,...
697387,A01,PRODUCT47,ITEM03451,201740,2017,-3
697571,A01,PRODUCT12,ITEM04038,201740,2017,-3
697922,A01,PRODUCT47,ITEM02923,201739,2017,-3
697923,A01,PRODUCT47,ITEM01138,201739,2017,-3


### 2-2. Dask

In [8]:
starttime = datetime.now()
targetColumns = ["REGIONID","PRODUCT","ITEM",\
                 "YEARWEEK","YEAR","QTY"]
selectedDataD = selloutDataD.loc\
                [ selloutDataD.QTY < 0 , targetColumns]

selectedDataD.compute()
timecheck(starttime)

time elapsed (hh:mm:ss.ms 0:00:02.051110)


# 3. 데이터 정제

### 3-1. pandas

In [9]:
starttime = datetime.now()
def refineQty(indata):
    if indata < 0:
        indata = 0
    return indata

selloutDataP["QTY_NEW"] =  selloutDataP.QTY.apply(refineQty)

selloutDataP
timecheck(starttime)

time elapsed (hh:mm:ss.ms 0:00:00.252325)


### 3-2. Dask

In [10]:
starttime = datetime.now()
def refineQty(indata):
    if indata < 0:
        indata = 0
    return indata

# apply function 사용시 새로운 데이터에 대한 타입 정의 필요
selloutDataD["QTY_NEW"] =  selloutDataD.QTY.apply(refineQty, 
                                                  meta=("NEW_QTY",'int64'))

selloutDataD
timecheck(starttime)
selloutDataD.compute()

time elapsed (hh:mm:ss.ms 0:00:00.013927)


,REGIONID,REGIONNAME,AP1ID,AP1NAME,ACCOUNTID,ACCOUNTNAME,SALESID,SALESNAME,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,TARGET,IDX,QTY_NEW
0,A01,KOPO,401737,EAST,SALESID0017,서울은행17지점,SALESID0017,서울은행17지점,PG4,PRODUCT12,ITEM02950,201728,2017,28,0,0.0,SALESID0017_ITEM02950,0
1,A01,KOPO,401737,EAST,SALESID0017,서울은행17지점,SALESID0017,서울은행17지점,PG4,PRODUCT12,ITEM01161,201728,2017,28,0,0.0,SALESID0017_ITEM01161,0
2,A01,KOPO,401737,EAST,SALESID0017,서울은행17지점,SALESID0017,서울은행17지점,PG4,PRODUCT12,ITEM02951,201728,2017,28,0,0.0,SALESID0017_ITEM02951,0
3,A01,KOPO,401737,EAST,SALESID0017,서울은행17지점,SALESID0017,서울은행17지점,PG4,PRODUCT12,ITEM04036,201728,2017,28,0,0.0,SALESID0017_ITEM04036,0
4,A01,KOPO,401737,EAST,SALESID0017,서울은행17지점,SALESID0017,서울은행17지점,PG4,PRODUCT12,ITEM02952,201728,2017,28,15,9.2,SALESID0017_ITEM02952,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259749,A01,KOPO,401737,EAST,SALESID0035,서울은행35지점,SALESID0035,서울은행35지점,PG4,PRODUCT12,ITEM00062,201740,2017,40,0,8.3,SALESID0035_ITEM00062,0
259750,A01,KOPO,401737,EAST,SALESID0035,서울은행35지점,SALESID0035,서울은행35지점,PG4,PRODUCT12,ITEM04049,201740,2017,40,-3,52.5,SALESID0035_ITEM04049,0
259751,A01,KOPO,401737,EAST,SALESID0035,서울은행35지점,SALESID0035,서울은행35지점,PG4,PRODUCT12,ITEM00011,201740,2017,40,0,0.0,SALESID0035_ITEM00011,0
259752,A01,KOPO,401737,EAST,SALESID0035,서울은행35지점,SALESID0035,서울은행35지점,PG4,PRODUCT12,ITEM03484,201740,2017,40,0,0.0,SALESID0035_ITEM03484,0


# 4. 데이터 집계

### 4-1. pandas

In [11]:
groupKey = ["PRODUCTGROUP"]
groupP = selloutDataP.groupby(groupKey)\
            [["QTY_NEW"]].agg("sum").reset_index()
groupP

,PRODUCTGROUP,QTY_NEW
0,PG1,24527603
1,PG2,1344540
2,PG3,2603010
3,PG4,3301972
4,PG5,794322


### 4-2. Dask

In [28]:
groupKey = ["PRODUCTGROUP"]
groupD = selloutDataD.groupby(groupKey)\
                [["QTY_NEW"]].agg("sum").reset_index()
groupD.compute()

,PRODUCTGROUP,QTY_NEW
0,PG1,24527603
1,PG2,1344540
2,PG3,2603010
3,PG4,3301972
4,PG5,794322


### 5. 데이터 조인

### 5-1. Pandas

In [13]:
joinKey = ["PRODUCTGROUP"]

joinDataP = pd.merge( left = selloutDataP, 
                     right=groupP,
         on=joinKey)
joinDataP.shape

(697996, 19)

### 5-2. Dask

In [14]:
joinKey = ["PRODUCTGROUP"]

joinDataD = dd.merge( left = selloutDataD, 
                     right=groupD,
         on=joinKey)
joinDataD.compute()

,REGIONID,REGIONNAME,AP1ID,AP1NAME,ACCOUNTID,ACCOUNTNAME,SALESID,SALESNAME,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,TARGET,IDX,QTY_NEW_x,QTY_NEW_y
0,A01,KOPO,401737,EAST,SALESID0017,서울은행17지점,SALESID0017,서울은행17지점,PG4,PRODUCT12,ITEM02950,201728,2017,28,0,0.0,SALESID0017_ITEM02950,0,3301972
1,A01,KOPO,401737,EAST,SALESID0017,서울은행17지점,SALESID0017,서울은행17지점,PG4,PRODUCT12,ITEM01161,201728,2017,28,0,0.0,SALESID0017_ITEM01161,0,3301972
2,A01,KOPO,401737,EAST,SALESID0017,서울은행17지점,SALESID0017,서울은행17지점,PG4,PRODUCT12,ITEM02951,201728,2017,28,0,0.0,SALESID0017_ITEM02951,0,3301972
3,A01,KOPO,401737,EAST,SALESID0017,서울은행17지점,SALESID0017,서울은행17지점,PG4,PRODUCT12,ITEM04036,201728,2017,28,0,0.0,SALESID0017_ITEM04036,0,3301972
4,A01,KOPO,401737,EAST,SALESID0017,서울은행17지점,SALESID0017,서울은행17지점,PG4,PRODUCT12,ITEM02952,201728,2017,28,15,9.2,SALESID0017_ITEM02952,15,3301972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259749,A01,KOPO,404706,RETAIL DIST,SALESID0117,부산은행26지점,SALESID0117,부산은행26지점,PG1,PRODUCT34,ITEM02398,201740,2017,40,0,0.0,SALESID0117_ITEM02398,0,24527603
259750,A01,KOPO,406417,B2B DISTRIBUTOR,SALESID0003,서울은행3지점,SALESID0003,서울은행3지점,PG1,PRODUCT34,ITEM00587,201740,2017,40,0,605.8,SALESID0003_ITEM00587,0,24527603
259751,A01,KOPO,406417,B2B DISTRIBUTOR,SALESID0003,서울은행3지점,SALESID0003,서울은행3지점,PG1,PRODUCT34,ITEM02992,201740,2017,40,60,19.6,SALESID0003_ITEM02992,60,24527603
259752,A01,KOPO,406417,B2B DISTRIBUTOR,SALESID0003,서울은행3지점,SALESID0003,서울은행3지점,PG1,PRODUCT34,ITEM00602,201740,2017,40,225,93.8,SALESID0003_ITEM00602,225,24527603


### 6. 정렬하기

### 6-1. Pandas

In [32]:
sortKey = ["QTY_NEW"]
groupP.sort_values(sortKey, 
                   inplace=True, 
                   ignore_index=True)
groupP

,PRODUCTGROUP,QTY_NEW
0,PG5,794322
1,PG2,1344540
2,PG3,2603010
3,PG4,3301972
4,PG1,24527603


### 6-2. Dask

In [29]:
sortKey = ["QTY_NEW"]
groupDD = groupD.set_index(sortKey)

groupDD = groupDD.reset_index()

groupDD.compute()

# Pandas Full Code

In [46]:
import pandas as pd

starttime = datetime.now()

### 0. 데이터 불러오기
selloutDataP = pd.read_csv\
        ("../dataset/kopo_channel_result_large.csv",
           encoding="ms949")

targetColumns = ["REGIONID","PRODUCT","ITEM",\
                 "YEARWEEK","YEAR","QTY"]
selectedDataP = selloutDataP.loc\
                [ selloutDataP.QTY < 0 , targetColumns]

def refineQty(indata):
    if indata < 0:
        indata = 0
    return indata

selloutDataP["QTY_NEW"] =  selloutDataP.QTY.apply(refineQty)

groupKey = ["PRODUCTGROUP"]
groupP = selloutDataP.groupby(groupKey)\
            [["QTY_NEW"]].agg("sum").reset_index()
groupP

joinKey = ["PRODUCTGROUP"]

joinDataP = pd.merge( left = selloutDataP, 
                     right=groupP,
         on=joinKey)

timecheck(starttime)
groupP

time elapsed (hh:mm:ss.ms 0:00:50.880604)


,PRODUCTGROUP,QTY_NEW
0,PG1,367914045
1,PG2,20168100
2,PG3,39045150
3,PG4,49529580
4,PG5,11914830


# Dask Full Code

In [47]:
import dask.dataframe as dd

starttime = datetime.now()

### 0. 데이터 불러오기
selloutDataD = dd.read_csv\
        ("../dataset/kopo_channel_result_large.csv",
           encoding="ms949")

targetColumns = ["REGIONID","PRODUCT","ITEM",\
                 "YEARWEEK","YEAR","QTY"]
selectedDataD = selloutDataD.loc\
                [ selloutDataD.QTY < 0 , targetColumns]

def refineQty(indata):
    if indata < 0:
        indata = 0
    return indata

# apply function 사용시 새로운 데이터에 대한 타입 정의 필요
selloutDataD["QTY_NEW"] =  selloutDataD.QTY.apply(refineQty, 
                                                  meta=("NEW_QTY",'int64'))

groupKey = ["PRODUCTGROUP"]
groupD = selloutDataD.groupby(groupKey)\
                [["QTY_NEW"]].agg("sum").reset_index()

joinKey = ["PRODUCTGROUP"]

joinDataD = dd.merge( left = selloutDataD, 
                     right=groupD,
         on=joinKey)

groupD.compute()
timecheck(starttime)

time elapsed (hh:mm:ss.ms 0:00:31.690720)
